In [24]:
!pip install nba_api pandas matplotlib scipy


In [25]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta

from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import players
import pandas as pd
from nba_api.stats.endpoints import playbyplayv2
from nba_api.stats.endpoints import playercareerstats


player_entered = str(input("Type Player Name (First and Last Name): "))
season_entered = int(input("Enter the year (Ex: if 2023-2024, then enter 2023): "))

def format_season(year):
    start = int(year)
    end = (start + 1) % 100
    return f"{start}-{end:02d}"

season_format = format_season(season_entered)

id_of_player = players.find_players_by_full_name(player_entered)[0]['id']

response = shotchartdetail.ShotChartDetail(player_id= id_of_player, team_id=0,  season_nullable= season_format, context_measure_simple= 'FGA')

shots = response.get_data_frames()[0]


game_ids = shots['GAME_ID'].unique()
print("Number of games found:", len(game_ids))



import time
from nba_api.stats.endpoints import playbyplayv2

data_for_each_game = []

def safe_get_pbp(game_id, retries=3, delay=2):
    for attempt in range(retries):
        try:
            pbp = playbyplayv2.PlayByPlayV2(game_id=game_id)
            return pbp.get_data_frames()[0]
        except Exception as e:
            print(f"Retry {attempt + 1} for game {game_id} failed: {e}")
            time.sleep(delay)
    print(f"Skipped game {game_id} after {retries} retries.")
    return None


def fill_missing_scoremargin(merged_df, pbp_df):

    #creates a dictionary; Asked for help for this line of code.
    score_lookup = pbp_df[['EVENTNUM', 'SCOREMARGIN']].dropna().set_index('EVENTNUM')['SCOREMARGIN'].to_dict()

    sorted_events = sorted(score_lookup.keys())
    filled = []

    for idx, row in merged_df.iterrows():
        if pd.notna(row['SCOREMARGIN']):
            filled.append(row['SCOREMARGIN'])
        else:
            current_event = row['GAME_EVENT_ID']
            previous = [ev for ev in sorted_events if ev < current_event]
            if previous:
                nearest = previous[-1]
                filled.append(score_lookup[nearest])
            else:
                filled.append('0')

    merged_df['SCOREMARGIN_FILLED'] = filled
    return merged_df


for game_id in game_ids:
    pbp_frame = safe_get_pbp(game_id)
    if pbp_frame is None:
        continue

    pbp_filtered = pbp_frame[['EVENTNUM', 'SCOREMARGIN']]
    shots_game = shots[shots['GAME_ID'] == game_id]

    merged = shots_game.merge(
        pbp_filtered,
        left_on='GAME_EVENT_ID',
        right_on='EVENTNUM',
        how='left'
    )

    merged = fill_missing_scoremargin(merged, pbp_frame)
    data_for_each_game.append(merged)
    time.sleep(0.5)

all_data = pd.concat(data_for_each_game, ignore_index=True)


# Convert SCOREMARGIN to numeric
all_data['SCOREMARGIN_NUM'] = pd.to_numeric(all_data['SCOREMARGIN_FILLED'], errors='coerce')


def is_clutch(row):
    return (
        row['PERIOD'] >= 4 and
        row['MINUTES_REMAINING'] <= 5 and
        abs(row['SCOREMARGIN_NUM']) <= 5
    )


all_data['IS_CLUTCH'] = all_data.apply(is_clutch, axis=1)


clutch_shots = all_data[all_data['IS_CLUTCH'] == True]
print(f'Your player took {len(clutch_shots)} clutch shots this season')


clutch_fg_pct = clutch_shots['SHOT_MADE_FLAG'].mean()
print(f"Clutch FG%: {clutch_fg_pct:.3f}")

def clutch_shot_value(row):
    if not row['IS_CLUTCH']:
        return 0

    if row['SHOT_TYPE'] == '3PT Field Goal':
        return 3 if row['SHOT_MADE_FLAG'] == 1 else -0.5
    else:
        return 2 if row['SHOT_MADE_FLAG'] == 1 else -1

all_data['CLUTCH_SHOT_VALUE'] = all_data.apply(clutch_shot_value, axis=1)


mean_clutch_value = all_data[all_data['IS_CLUTCH'] == True]['CLUTCH_SHOT_VALUE'].mean()
print(f"Mean Clutch Shot Value: {mean_clutch_value:.3f}")

def categorize_clutch_score(value):
    if value >= 0.7:
        return "Elite"
    elif value >= 0.5:
        return "Above Average"
    elif value >= 0.3:
        return "Average"
    elif value >= 0.1:
        return "Below Average"
    else:
        return "Liability"
print("Based off your player's mean clutch value...")
print(f'Your player is a {categorize_clutch_score(mean_clutch_value)} clutch player')















Type Player Name (First and Last Name): Stephen Curry
Enter the year (Ex: if 2023-2024, then enter 2023): 2015
Number of games found: 79
Your player took 79 clutch shots this season
Clutch FG%: 0.430
Mean Clutch Shot Value: 0.696
Based off your player's mean clutch value...
Your player is a Above Average clutch player
